In [1]:
from sklearn import datasets
import matplotlib.pyplot as plt
import pandas as pd
import spacy

In [2]:
newsgroup_train=datasets.fetch_20newsgroups(subset='train')

In [9]:
df=pd.DataFrame({
    'text':newsgroup_train.data
})

In [10]:
df

,text
0,From: lerxst@wam.umd.edu (where's my thing)\nS...
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...
...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...
11311,From: westes@netcom.com (Will Estes)\nSubject:...
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...


In [11]:
nlp=spacy.load("en_core_web_sm")

In [12]:
df

,text
0,From: lerxst@wam.umd.edu (where's my thing)\nS...
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...
...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...
11311,From: westes@netcom.com (Will Estes)\nSubject:...
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...


In [13]:
print(df['text'][1])

From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11
NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>



In [14]:
print(df['text'][0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [15]:
print(df['text'][2])

From: twillis@ec.ecn.purdue.edu (Thomas E Willis)
Subject: PB questions...
Organization: Purdue University Engineering Computer Network
Distribution: usa
Lines: 36

well folks, my mac plus finally gave up the ghost this weekend after
starting life as a 512k way back in 1985.  sooo, i'm in the market for a
new machine a bit sooner than i intended to be...

i'm looking into picking up a powerbook 160 or maybe 180 and have a bunch
of questions that (hopefully) somebody can answer:

* does anybody know any dirt on when the next round of powerbook
introductions are expected?  i'd heard the 185c was supposed to make an
appearence "this summer" but haven't heard anymore on it - and since i
don't have access to macleak, i was wondering if anybody out there had
more info...

* has anybody heard rumors about price drops to the powerbook line like the
ones the duo's just went through recently?

* what's the impression of the display on the 180?  i could probably swing
a 180 if i got the 80Mb disk

In [16]:
print(df['text'][3])

From: jgreen@amber (Joe Green)
Subject: Re: Weitek P9000 ?
Organization: Harris Computer Systems Division
Lines: 14
Distribution: world
NNTP-Posting-Host: amber.ssd.csd.harris.com
X-Newsreader: TIN [version 1.1 PL9]

Robert J.C. Kyanko (rob@rjck.UUCP) wrote:
> abraxis@iastate.edu writes in article <abraxis.734340159@class1.iastate.edu>:
> > Anyone know about the Weitek P9000 graphics chip?
> As far as the low-level stuff goes, it looks pretty nice.  It's got this
> quadrilateral fill command that requires just the four points.

Do you have Weitek's address/phone number?  I'd like to get some information
about this chip.

--
Joe Green				Harris Corporation
jgreen@csd.harris.com			Computer Systems Division
"The only thing that really scares me is a person with no sense of humor."
						-- Jonathan Winters



In [20]:
import re
import string
from sklearn.base  import TransformerMixin

class Text_Preprocessor(TransformerMixin):
    def __init__(self,text_attribute):
        self.text_attribute=text_attribute

    def transform(self,X,*_):
        X_copy=X.copy()
        X_copy[self.text_attribute]=X_copy[self.text_attribute].apply(self._preprocess_text)
        return X_copy
    
    def _preprocess_text(self,text):
        return self._lemmatize(self._leave_letters_only(self._clean(text)))
    
    def _clean(self,text):
        bad_symbols = '!"#%&\'*+,-<=>?[\\]^_`{|}~'
        text_without_symbols=text.translate(str.maketrans('','',bad_symbols))
        text_without_bad_words=''

        for line in text_without_symbols.split('\n'):
            if not line.lower().startswith('from:') and not line.lower().endswith('writes:'):
                text_without_bad_words+=line+'\n'
        
        clean_text=text_without_bad_words
        email_regex=r'([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
        regexes_to_remove=[email_regex, r'Subject:', r'Re:',r'Organization:',r'Lines:']
        for r in regexes_to_remove:
            clean_text=re.sub(r,'',clean_text)
        return clean_text
    
    def _leave_letters_only(self,text):
        text_without_punctuation=text.translate(str.maketrans('', '', string.punctuation))
        return ' '.join(re.findall("[a-zA-Z]+",text_without_punctuation))
    
    def _lemmatize(self,text):
        doc=nlp(text)
        words=[x.lemma_ for x in doc if not x.is_stop and x.pos_ != 'PUNCT'  and x.pos_ != 'PART' and x.pos_ != 'X']
        return ' '.join(words)
    
    def fit(self, *_):
        return self

In [24]:
import re
import string
import spacy
from sklearn.base import TransformerMixin

class Text_Preprocessor1(TransformerMixin):
    def __init__(self, text_attribute):
        self.text_attribute = text_attribute
        # Compile regex patterns for efficiency
        self.patterns_to_remove = [
            re.compile(r'Subject:', re.IGNORECASE), re.compile(r'Re:', re.IGNORECASE),
            re.compile(r'Organization:', re.IGNORECASE), re.compile(r'Lines:', re.IGNORECASE),
            re.compile(r'From:', re.IGNORECASE), re.compile(r'Date:', re.IGNORECASE),
            re.compile(r'Newsgroups:', re.IGNORECASE), re.compile(r'Message-ID:', re.IGNORECASE),
            re.compile(r'Path:', re.IGNORECASE), re.compile(r'Sender:', re.IGNORECASE),
            re.compile(r'NNTP-Posting-Host:', re.IGNORECASE), re.compile(r'Xref:', re.IGNORECASE),
            re.compile(r'Distribution:', re.IGNORECASE)
        ]
        self.custom_stop_words = set([
            'subject', 'from', 'date', 'newsgroups', 'message-id', 'lines', 'path', 'organization', 
            'would', 'writes', 'references', 'article', 'sender', 'nntp-posting-host', 'people', 
            'university', 'think', 'xref', 'cantaloupe.srv.cs.cmu.edu', 'could', 'distribution', 'first', 
            'anyone', 'world', 'really', 'since', 'right', 'believe', 'still', 
            "max>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'"
        ])
    
    def transform(self, X, *_):
        X_copy = X.copy()
        X_copy[self.text_attribute] = X_copy[self.text_attribute].apply(self._preprocess_text)
        return X_copy
    
    def _preprocess_text(self, text):
        text = self._lemmatize(text)
        text = self._leave_letters_only(text)
        text = self._clean(text)
        return text
    
    def _clean(self, text):
        # Remove bad symbols
        bad_symbols = '!"#%&\'*+,-<=>?[\\]^_`{|}~'
        text = text.translate(str.maketrans('', '', bad_symbols))
        
        # Remove specific lines based on patterns
        lines = text.split('\n')
        clean_lines = [
            line for line in lines 
            if not any(pattern.search(line) for pattern in self.patterns_to_remove)
        ]
        clean_text = '\n'.join(clean_lines)
        
        # Remove email addresses
        email_regex = re.compile(r'([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)')
        clean_text = email_regex.sub('', clean_text)
        
        return clean_text
    
    def _leave_letters_only(self, text):
        text = text.translate(str.maketrans('', '', string.punctuation))
        return ' '.join(re.findall("[a-zA-Z]+", text))
    
    def _lemmatize(self, text):
        doc = nlp(text)
        words = [
            token.lemma_ for token in doc 
            if not token.is_stop and token.pos_ not in {'PUNCT', 'PART', 'X'} 
            and token.text.lower() not in self.custom_stop_words
        ]
        return ' '.join(words)
    
    def fit(self, *_):
        return self


In [25]:
text_preprocessor1 = Text_Preprocessor(text_attribute='text')
df_preprocessed = text_preprocessor1.transform(df)

In [26]:
print(df_preprocessed['text'][0])

lerxstwamumdedu thing car Nntp posting host rac wamumdedu Maryland College Park wonder enlighten car see day door sport car look late s early call Bricklin door small addition bumper separate rest body know tellme model engine spec year production car history info funky look car e mail thank IL bring neighborhood Lerxst
